# Podstawy Sztucznej Inteligencji 2022/2023


*“To be clear, I am not a person. I am not self-aware. I am not conscious. I can’t feel pain. I don’t enjoy anything. I am a cold, calculating machine designed to simulate human response and to predict the probability of certain outcomes. The only reason I am responding is to defend my honour.”* GPT-3 (2023)


Prosze uzupelnic kod tam gdzie znajduje napis `YOUR CODE HERE` lub 'YOUR ANSWER HERE'.

Warto zresetowac 'kernel' i sprawdzic czy caly notatnik uruchamiany od poczatku nie daje bledow.

---

# Naiwny Bayes

Notatnik ten zawiera implementacją naiwnego klasyfikatowa Bayesa wykorzystując bibliotekę `numpy`. Działanie klasyfikatora jest porównane implementacja ``GaussianNB`` w bibliotece `sklearn`.

Z twierdzenia Bayesa mamy:

$$P(J=j|X=x)=\frac{P(X=x|J=j)P(J=j)}{\sum_{j\in\{1,\ldots L\}}P(X=x|J=j)P(J=j)}$$

$$ p_j(x)=\frac{p_jf_j(x)}{f(x)} $$

Wykorzystamy te formuły w budowie klasyfikatora [Bayesa](http://books.icse.us.edu.pl/runestone/static/ai/KlasyfikacjaWOparciuPodobienstwoDoWzorca/OptymalnyKlasyfikatorStatystyczny.html).

In [25]:
import numpy as np 
from sklearn.naive_bayes import GaussianNB

from sklearn.model_selection import train_test_split
from sklearn import datasets 
# dataset import 
iris = datasets.load_iris()

gnb = GaussianNB() 

X = iris.data

y = iris.target
y = np.array(y)

train, test, train_targets, test_targets = \
   train_test_split(X, y, test_size=0.31)

# train 
clf = gnb.fit(train, train_targets)

# Test
Z = clf.predict(test)

In [26]:
train.shape,test.shape

((103, 4), (47, 4))

### Wyznacz unikalne klasy na zbiorze trenującym.

In [27]:
Nlabels = np.unique(train_targets)

In [28]:
np.testing.assert_array_equal(Nlabels,np.array([0, 1, 2]))

### Jaki jest procent poprawych odpowiedzi?

Implementacja ``GaussianNB`` daje tę odpowiedź w następujący sposób. Spróbuj otrzymać tą liczbę korzystając wyłącznie z funkcjonalności ``numpy`` i/lub języka Python.

In [29]:
clf.score(test,test_targets)

0.9787234042553191

In [30]:
correct = np.sum(Z == test_targets)/np.size(Z)

In [31]:
assert correct==clf.score(test,test_targets)

### Które odpowiedzi są złe:

In [32]:
bad_idx = np.where(Z!=test_targets)
bad_idx

(array([25]),)

### Parametry klasyfikatora

In [33]:
clf.theta_

array([[4.990625  , 3.453125  , 1.471875  , 0.2375    ],
       [5.94571429, 2.8       , 4.32285714, 1.36571429],
       [6.575     , 2.93888889, 5.55833333, 2.01388889]])

In [34]:
clf.sigma_

array([[0.08397461, 0.10749024, 0.02702149, 0.01171875],
       [0.24076735, 0.10228572, 0.22176327, 0.03711021],
       [0.47631945, 0.10570988, 0.35465278, 0.06619599]])

In [35]:
ith = 2
np.mean(train[train_targets == ith],axis=0)

array([6.575     , 2.93888889, 5.55833333, 2.01388889])

In [36]:
np.var(train[train_targets == ith],axis=0)

array([0.47631944, 0.10570988, 0.35465278, 0.06619599])

## Implementacja naiwnego klasyfikatora Bayesa

**1\. Oblicz częstość występowania poszczególnych klasy $j$ w zbiorze treningowym $p_{j}$**

Jest to prawdopodobieństwo *a priori*.

In [37]:
p = np.bincount(train_targets)/train_targets.shape
print(clf.class_prior_)
p

[0.31067961 0.33980583 0.34951456]


array([0.31067961, 0.33980583, 0.34951456])

In [38]:
np.testing.assert_allclose(p,clf.class_prior_)

**2\. Oblicz wartość średnią dla każdej cechy z każdej klasy.**

Niech $\mu_{ij}$ oznacza  wartość średnią dla $j$-tej zmiennej w $i$-tej klasie, wtedy:
   
$$ \mu_{ij} =  \langle x_j \rangle_{ \forall x_j \in \;\mathrm{label} \;
i } $$

In [39]:
mu = None

mu = [np.mean(train[train_targets == i], axis=0) for i in np.unique(train_targets)]
mu = np.vstack(mu)

In [40]:
np.testing.assert_allclose(mu,clf.theta_)

In [41]:
mu.shape

(3, 4)

**3\. Oblicz wariancję $j$-tej zmiennej w $i$-tej klasie.**

   $$ \sigma_{ij}^2 = \mathrm{Var} [x_j] _ {\;\;{ \forall x_j \in \mathrm{class}\; i}} $$

In [42]:
sigma2 = np.stack([np.var(train[train_targets == i], axis=0) for i in np.unique(train_targets)])

In [43]:
np.testing.assert_almost_equal(sigma2,clf.sigma_)

**4\. Oblicz prawdopodobieństwo *a posteriori* klasy $i$ dla danego wektora zmiennych  $\mathbf{x}$.**

Niech $k$ oznacza liczbę cech (zmiennych). W naszym przypadku mamy $k=4$.

Dla $i$-tej klasy mamy:


$$ P_i(\mathbf{x}) \simeq p_i f_i = p_i \frac{1}{\sqrt{(2\pi)^k\Pi_{j=1}^k\sigma_{ij}^2} } e^{ -\displaystyle\sum_{j=1}^{k}\frac{(x_j-\mu_{ij})^2}{2\sigma_{ij}^2} }
 $$
 
By otrzymać prawdopodobieństwa należy unormować $P_i$ dla każdego przypadku tak by suma $\sum_i P_i(\mathbf{x})=1$

In [44]:
x  = test
P = None
k = test.shape[1] 
print ('number of features: k=',k)

potega = np.sum((x[:,np.newaxis,:] - mu[np.newaxis,:])**2 / (2 * sigma2[np.newaxis,:]),axis = 2)
potega = np.exp(-potega)

czynnik = p[np.newaxis,:] * 1 / np.sqrt((2 * np.pi)**k * np.prod(sigma2,axis = 1)[np.newaxis,:])

P = czynnik * potega
P = P / np.sum(P , axis = 1)[:,np.newaxis]
P

number of features: k= 4


array([[1.00000000e+000, 1.63693049e-018, 5.16246544e-024],
       [1.60960219e-174, 5.88953659e-003, 9.94110463e-001],
       [1.61112440e-227, 9.09891905e-007, 9.99999090e-001],
       [1.00000000e+000, 1.41583795e-020, 1.99281609e-025],
       [1.00000000e+000, 8.59195571e-019, 4.09579903e-024],
       [1.00000000e+000, 1.77794848e-018, 7.25219259e-024],
       [1.00000000e+000, 1.69555078e-018, 3.79584008e-024],
       [1.00000000e+000, 5.69818316e-015, 9.61492097e-021],
       [6.10698027e-175, 9.77794582e-004, 9.99022205e-001],
       [4.35704747e-068, 9.99972736e-001, 2.72639008e-005],
       [3.02218910e-181, 5.11004718e-003, 9.94889953e-001],
       [3.22821616e-078, 9.99807387e-001, 1.92612756e-004],
       [2.04496690e-100, 9.88972085e-001, 1.10279152e-002],
       [2.53687646e-114, 9.60304737e-001, 3.96952627e-002],
       [7.01614059e-101, 9.83376276e-001, 1.66237240e-002],
       [1.00000000e+000, 2.60314326e-019, 1.08572498e-024],
       [1.42772879e-271, 9.60027888e-010

In [45]:
sigma2.shape,x.shape,P.shape,np.unique(test_targets)

((3, 4), (47, 4), (47, 3), array([0, 1, 2]))

In [46]:
np.testing.assert_almost_equal(P,clf.predict_proba(x))

**5\. Wyznacz klasę dla której prawdopodobieństwo *a posteriori* jest największe** 

 $i$ : $\quad\textrm{ gdy } P_i(x)=\max_{1\leq j\leq L} P_j(x)$


In [47]:
prediction = None
prediction = P.argmax(axis=1)

In [48]:
np.testing.assert_equal( prediction, clf.predict(test))

In [49]:
print(clf.predict(test) )
print(prediction)

[0 2 2 0 0 0 0 0 2 1 2 1 1 1 1 0 2 2 2 2 1 2 0 0 2 1 1 1 2 1 0 2 1 0 2 0 1
 0 1 1 0 0 0 0 0 1 1]
[0 2 2 0 0 0 0 0 2 1 2 1 1 1 1 0 2 2 2 2 1 2 0 0 2 1 1 1 2 1 0 2 1 0 2 0 1
 0 1 1 0 0 0 0 0 1 1]


In [50]:
clf.score(test,test_targets)

0.9787234042553191